Run the next cell

you need T4 gpu to be able to test the model

In [ ]:
!nvidia-smi -L

In [ ]:
#@title Install Tacotron and Waveglow (click to see code)

import os
from os.path import exists, join, basename, splitext
!pip install gdown
git_repo_url = 'https://github.com/maxmelichov/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 2fd4e63
  !pip install -q librosa unidecode
  !pip install Hebrew
  
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
import gdown
from hebrew import Hebrew
from hebrew.chars import HebrewChar, ALEPH
from hebrew import GematriaTypes
d = 'https://drive.google.com/uc?id='

The pre-trained model

In [ ]:
#@title Install Tacotron pretrained model
force_download_TT2 = True
tacotron2_pretrained_model = 'MLPTTS'
if not exists(tacotron2_pretrained_model) or force_download_TT2:
                   # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ PUT MODEL HERE
  gdown.download(d+r'1ByWFnQsqIZuJpz7PgBgCo4ED63eFyKKp&export=download', tacotron2_pretrained_model, quiet=False); print("Tacotron2 Model Downloaded")
                   # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑ PUT MODEL HERE



In [ ]:
#@title Install Waveglow pretrained
waveglow_pretrained_model = 'waveglow.pt'
if not exists(waveglow_pretrained_model):
  gdown.download(d+r'1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF&export=download', waveglow_pretrained_model, quiet=False); print("WaveGlow Model Downloaded")#1okuUstGoBe_qZ4qUEF8CcwEugHP7GM_b&export

In [ ]:
!python ./tacotron2/waveglow/convert_model.py /content/waveglow.pt /content/waveglow.pt

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding()

In [ ]:
#@title Initialize Tacotron and Waveglow and Hebrew to English fucntion
%matplotlib inline
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

graph_width = 900
graph_height = 360
def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
    %matplotlib inline
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='upper', 
                       interpolation='none', cmap='inferno')
    fig.canvas.draw()
    plt.show()



import re
 
 
##breaks down number to digits
def breakdown(number):
    digits = []
    for place, value in zip([1000, 100, 10, 1], [1000, 100, 10, 1]):
        digit = number // place * value
        digits.append(digit)
        number -= digit
    return digits
 
##turns number to hebrew
def NumberToHebrew(number):
 
    if number==0:
        return "אֶפֶס"
 
    num_dict_below_20={
        1: 'אֶחָד',
        2: 'שְׁנַיִם',
        3: 'שְׁלֹשָׁה',
        4: 'אַרְבָּעָה',
        5: 'חֲמִשָּׁה',
        6: 'שִׁשָּׁה',
        7: 'שִׁבְעָה',
        8: 'שְׁמוֹנָה',
        9: 'תִּשְׁעָה',
        10: 'עֶשֶׂר',
        11: 'אַחַד עָשָׂר',
        12: 'שְׁנֵים עָשָׂר',
        13: 'שְׁלֹשָׁה עָשָׂר',
        14: 'אַרְבָּעָה עָשָׂר',
        15: 'חֲמִשָּׁה עָשָׂר',
        16: 'שִׁשָּׁה עָשָׂר',
        17: "שִׁבְעָה עֶשְׂרֵה",
        18: "שְׁמוֹנָה עֶשְׂרֵה",
        19: "תִּשְׁעָה עֶשְׂרֵה"
    }
 
    num_dict_eq_above_20={
        20: "עֶשְׂרִים",
        30: "שְׁלשִׁים",
        40: "אַרְבָּעִים",
        50: "חֲמִשִּׁים",
        60: "שִׁשִּׁים",
        70: "שִׁבְעִים",
        80: "שְׁמוֹנִים",
        90: "תִּשְׁעִים",
        100: "מֵאָה",
        200: "מָאתַיִם",
        300: "שְׁלֹשׁ מֵאוֹת",
        400: "אַרְבָּעִ מֵאוֹת",
        500: "חֲמִשֶּׁ מֵאוֹת",
        600: "שֵׁשׁ מֵאוֹת",
        700: "שִׁבְעַ מֵאוֹת",
        800: "שְׁמוֹנֶ מֵאוֹת",
        900: "תִּשְׁעַ מֵאוֹת",
        1000: "אֶלֶף",
        2000: "אֲלַפַּיִם",
        3000: "שְׁלֹשֶׁת אֲלָפִים",
        4000: "אַרְבַּעַת אֲלָפִים",
        5000: "חֲמֵשׁ אֲלָפִים",
        6000: "שֵׁשׁ אֲלָפִים",
        7000: "שִׁבְעָה אֲלָפִים",
        8000: "שְׁמוֹנָה אֲלָפִים",
        9000: "תִּשְׁעָה אֲלָפִים"
    }
 
    list_heb=[]
    list_num=breakdown(number)
 
    for i,num in enumerate(list_num):
        if num==0:
            continue
        else:
            if i<2:
                list_heb.append(num_dict_eq_above_20[num])
 
            elif i==2:
                if list_num[0]!=0 or list_num[1]!=0:
                    if list_num[2]+list_num[3]<20:
                        list_heb.append("וְ"+num_dict_below_20[list_num[2]+list_num[3]])
                        break
                    else:
                        list_heb.append(num_dict_eq_above_20[num])
                else:
                    if list_num[2] + list_num[3] < 20:
                        list_heb.append(num_dict_below_20[list_num[2] + list_num[3]])
                        break
                    else:
                        list_heb.append(num_dict_eq_above_20[num])
 
            elif i==3:
                if list_num[0]!=0 or list_num[1]!=0 or list_num[2]!=0:
                    list_heb.append("וְ" + num_dict_below_20[num])
                else:
                    list_heb.append(num_dict_below_20[num])
 
    return list_heb
 
 
##breaks text to list
def break_to_list(text):
    """
    This function receives a string and returns a list of strings with each word from the input text.
    """
    lst = []
    for tav in text:
        lst.append(tav)
    return lst
 
##takes a letter in hebrew and returns the sound in english
def English(obj,tzuptzik,last_letter=False):
    obj = Hebrew(obj).string
    # map the nikud symbols to their corresponding phenoms
    nikud_map = {"ָ": "a", "ַ": "a", "ֶ": "e", "ֵ": "e", "ִ": "i", "ְ": "", "ֹ": "o", "ֻ": "oo", 'ּ': "", 'ֲ': 'a'}
    # fixme need create more hebrew phenoms
 
    beged_kefet_shin_sin = {
        ############ B
        "בּ": "b",
        "בְּ": "b",
        "בִּ": "bi",
        "בֹּ": "bo",
        "בֵּ": "be",
        "בֶּ": "be",
        "בַּ": "ba",
        "בָּ": "ba",
        "בֻּ": "boo",
        ############ G
        "גּ": "g",
        "גְּ": "g",
        "גִּ": "gi",
        "גֹּ": "go",
        "גֵּ": "ge",
        "גֶּ": "ge",
        "גַּ": "ga",
        "גָּ": "ga",
        "גֻּ": "goo",
        ########### D
        "דּ": "d",
        "דְּ": "d",
        "דִּ": "di",
        "דֹּ": "do",
        "דֵּ": "de",
        "דֶּ": "de",
        "דַּ": "da",
        "דָּ": "da",
        "דֻּ": "doo",
        ########### K
        "כּ": "k",
        "כְּ": "k",
        "כִּ": "ki",
        "כֹּ": "ko",
        "כֵּ": "ke",
        "כֶּ": "ke",
        "כַּ": "ka",
        "כָּ": "ka",
        "כֻּ": "koo",
        ############ P
        "פּ": "p",
        "פְּ": "p",
        "פִּ": "pi",
        "פֹּ": "po",
        "פֵּ": "pe",
        "פֶּ": "pe",
        "פַּ": "pa",
        "פָּ": "pa",
        "פֻּ": "poo",
        ############ T
        "תּ": "t",
        "תְּ": "t",
        "תִּ": "ti",
        "תֹּ": "to",
        "תֵּ": "te",
        "תֶּ": "te",
        "תַּ": "ta",
        "תָּ": "ta",
        "תֻּ": "too",
        ############ S
        "שׂ": "s",
        "שְׂ": "s",
        "שִׂ": "si",
        "שֹׂ": "so",
        "שֵׂ": "se",
        "שֶׂ": "se",
        "שַׂ": "sa",
        "שָׂ": "sa",
        "שֻׂ": "soo",
        ########### SH
        "שׁ": "sh",
        "שְׁ": "sh",
        "שִׁ": "shi",
        "שֹׁ": "sho",
        "שֵׁ": "she",
        "שֶׁ": "she",
        "שַׁ": "sha",
        "שָׁ": "sha",
        "שֻׁ": "shoo",
    }
 
    vav = {
        "וֵּו": "ve",
        "וּ": "oo",
        "וּו": "oo",
        "וֹ": "o",
        "וֹו": "oo",
        "וְ": "ve",
        "וֱו": "ve",
        "וִ": "vi",
        "וִו": "vi",
        "וַ": "va",
        "וַו": "va",
        "וֶ": "ve",
        "וֶו": "ve",
        "וָ": "va",
        "וָו": "va",
        "וֻ": "oo",
        "וֻו": "oo"
    }
 
 
    letters_map = {
        "א": "",
        "ב": "v",
        "ג": "g",
        "ד": "d",
        "ה": "hh",
        "ו": "v",
        "ז": "z",
        "ח": "h",
        "ט": "t",
        "י": "y",
        "כ": "h",
        "ל": "l",
        "מ": "m",
        "נ": "n",
        "ס": "s",
        "ע": "",
        "פ": "f",
        "צ": "ts",
        "ק": "k",
        "ר": "r",
        "ש": "sh",
        "ת": "t",
        "ן": "n",
        "ם": "m",
        "ף": "f",
        "ץ": "ts",
        "ך": "h",
    }
 
    patah_ganav={
        "חַ": "ah",
        "חָ": "ah",
        "הַ": "hha",
        "הָ": "hha",
        "עַ": "a",
        "עָ": "a",
 
    }
 
    tzuptzik_letters={
        ##G
        "ג": "j",
        "גְ": "j",
        "גִ": "ji",
        "גֹ": "jo",
        "גֵ": "je",
        "גֶ": "je",
        "גַ": "ja",
        "גָ": "ja",
        "גֻ": "joo",
        "גּ": "j",
        "גְּ": "j",
        "גִּ": "ji",
        "גֹּ": "jo",
        "גֵּ": "je",
        "גֶּ": "je",
        "גַּ": "ja",
        "גָּ": "ja",
        "גֻּ": "joo",
 
        ##ch
        "צ": "ch",
        "צְ": "ch",
        "צִ": "chi",
        "צֹ": "cho",
        "צֵ": "che",
        "צֶ": "che",
        "צַ": "cha",
        "צָ": "cha",
        "צֻ": "choo",
 
        ##Z
        "ז": "zh",
        "זְ": "zh",
        "זִ": "zhi",
        "זֹ": "zho",
        "זֵ": "zhe",
        "זֶ": "zhe",
        "זַ": "zha",
        "זָ": "zha",
        "זֻ": "zhoo",
    }
 
    if last_letter:
        if obj in patah_ganav:
            return patah_ganav[obj]
 
    if tzuptzik==True:
        if obj in tzuptzik_letters:
            return tzuptzik_letters[obj]
 
    if obj in beged_kefet_shin_sin:
        return beged_kefet_shin_sin[obj]
    elif obj in vav:
        return vav[obj]
    else:
        lst = break_to_list(obj)
        string = ""
        for item in lst:
            if item in letters_map:
                string += letters_map[item]
            if item in nikud_map:
                string += nikud_map[item]
 
        return string
 
 
##takes hebrew word and turns it into the sound in english
def HebWordToEng(word,index):
    new_sentence=""
    hs = Hebrew(word)
    hs = Hebrew(list(hs.graphemes)).string
    for i, letter in enumerate(hs):
 
        tzuptzik = False
        if i < len(hs) - 1:
            if hs[i + 1] == '\'':
                tzuptzik = True
 
        tav = English(letter, tzuptzik, i == len(hs) - 1)
        new_sentence += tav
 
    ##clean list:
    try:
        if new_sentence[-1] == 'y' and new_sentence[-2] == 'y':
            new_sentence = new_sentence.replace("yy", "y")
    except:
       pass
    return new_sentence
 
##takes hebrew sentence and turns it into english sounds
def ARPA(sentence,index=0):
    words = sentence.split()
    new_sentence = ""
 
    for word in words:
        ##if number not in string
        if not has_number(word):
            ret_sentence=HebWordToEng(word,index)
            new_sentence+=ret_sentence+" "
 
        ##if there is a number:
        else:
            try:
                before_num,num,after_num=split_number_and_string(word)
 
                if has_number(after_num) or has_number(before_num):
                    raise(Exception)
 
 
                ret_sentence = HebWordToEng(before_num, index)
                new_sentence += ret_sentence+" "
 
                num = [s for s in word if s.isdigit()]
                num="".join(num)
                num=int(num)
                list_of_numbers=NumberToHebrew(num)
                for number in list_of_numbers:
                    ret_sentence=HebWordToEng(number,index)
                    new_sentence += ret_sentence + " "
 
                ret_sentence = HebWordToEng(after_num, index)
                new_sentence += ret_sentence + " "
            except:
                print(f"error from split_number_and_string in line:{index} ,the word is {word}" )
 
 
    new_sentence = ' '.join(new_sentence.split())
    return new_sentence
 
 
def split_number_and_string(input_string):
    # Use regular expression to find any number within the string
    match = re.search(r'\d+', input_string)
 
    if match:
        # Extract the number from the string
        number = match.group()
 
        # Split the string into two parts: before and after the number
        index = match.start()
        string_before_number = input_string[:index]
        string_after_number = input_string[index + len(number):]
 
        return string_before_number, number, string_after_number
    else:
        # If no number is found, return None
        return None
 
 
def has_number(input_string):
    # Use regular expression to search for any digits within the string
    return bool(re.search(r'\d', input_string))
 

 
 



torch.set_grad_enabled(False)

# initialize Tacotron2 with the pretrained model
hparams = create_hparams()

In [ ]:
# Load Tacotron2 (run this cell every time you change the model)
hparams.sampling_rate = 22050 # Don't change this
hparams.max_decoder_steps = 1000 # How long the audio will be before it cuts off (1000 is about 11 seconds)
hparams.gate_threshold = 0.1 # Model must be 90% sure the clip is over before ending generation (the higher this number is, the more likely that the AI will keep generating until it reaches the Max Decoder Steps)
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval()

In [ ]:
# Load WaveGlow
waveglow = torch.load(waveglow_pretrained_model)['model']
waveglow.cuda().eval()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
text = "בָּנַיי"
sigma = 0.8
denoise_strength = 0.1
# try to switch raw data to True maybe the results will be better
raw_input = False # disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing

for i in text.split("\n"):
    if len(i) < 1: continue;
    print(i)
    if raw_input:
        if i[-1] != ";": i=i+";" 
    else: i = ARPA(i)
    print(i)
    with torch.no_grad(): # save VRAM by not including gradients
        sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
        sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
        mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
        plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],alignments.float().data.cpu().numpy()[0].T))
        audio = waveglow.infer(mel_outputs_postnet, sigma=sigma); print(""); ipd.display(ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate))